In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/others/julia`


In [2]:
using ThreadPools
using DataStructures
using GLMakie

# Wait, notify demo - (communication using channels)

In [3]:
c1 = Channel{Float64}(10)

Channel{Float64}(10) (empty)

### Receiver (Run first)

In [4]:
# new condition created each time (becuase contions remains locked after use for some reason)
condition = Threads.Condition()

receiver = @tspawnat 2 begin
    Core.println("Waiting for data")

    for i = 1:10

        # wait for data
        lock(condition)
        wait(condition)

        # Do stuff
        data = take!(c1)
        Core.println("Received data", data)

        # notify sender that stuff has been completed
        lock(condition)
        notify(condition)
    end

end

Task (runnable) @0x000000016a2f2b60

Waiting for data


### Sender

In [5]:
sender = @tspawnat 1 begin
    for i = 1:10

        # send data
        put!(c1, i)
        Core.println("Sent data", i)

        lock(condition)
        notify(condition)

        # wait for received to complete stuff
        lock(condition)
        wait(condition)
    end

end

Sent data1


Task (runnable) @0x000000016d675f90

Received data1
Sent data2
Received data2
Sent data3
Received data3
Sent data4
Received data4
Sent data5
Received data5
Sent data6
Received data6
Sent data7
Received data7
Sent data8
Received data8
Sent data9
Received data9
Sent data10
Received data10


# Wait, notify demo - (communication without channels, using just notify)

### Receiver (Run first)

In [146]:
# new condition created each time (becuase contions remains locked after use for some reason)
condition = Threads.Condition()

receiver = @tspawnat 2 begin

    # if islocked(condition)
    #     unlock(condition)
    # end
    
    Core.println("Waiting for data")

    for i = 1:10

        # wait for data
        lock(condition)
        data = wait(condition)

        # Do stuff
        Core.println("Received data", data)

        # notify sender that stuff has been completed
        lock(condition)
        notify(condition)
    end
    unlock(condition)
     unlock(condition)
end

Task (runnable) @0x000000017466c400

Waiting for data


### Sender

In [145]:
sender = @tspawnat 1 begin
    for i = 1:10

         # send data
        lock(condition)
        notify(condition, i)
        
        Core.println("Sent data", i)

        # wait for received to complete stuff
        lock(condition)
        wait(condition)
    end
end

Sent data1
Received data1
Sent data2
Received data2
Sent data3
Received data3

Task (runnable) @0x0000000174666cb0


Sent data4
Received data4
Sent data5
Received data5
Sent data6
Received data6
Sent data7
Received data7
Sent data8
Received data8
Sent data9
Received data9
Sent data10
Received data10


## Simple notify, wait demo

In [221]:
condition = Condition()

@async begin for i=1:10
    wait(condition)
    Core.println("Selva: $i") 
    end
end

for i=1:10
    notify(condition)
    sleep(0.1)
end

Selva: 1
Selva: 2
Selva: 3
Selva: 4
Selva: 5
Selva: 6
Selva: 7
Selva: 8
Selva: 9


# Receiver , circular buffer demo

In [65]:
function sender(c1)
    Core.println("Waiting to send data")
    for i = 1:10
        # send data
        put!(c1, i)
        # Core.println("Sent data:", i)
        
        sleep(0.1)
    end
end


function receiver(c1, flag)
    Core.println("Waiting to receive data")
    buffer_len = 10
    
    buffer = CircularDeque{Float64}(buffer_len)  
    #buffer = CircularBuffer{Float64}(buffer_len)  

    first_received_flag = false
    

    @async begin while flag[]==true
        # send data
        data = take!(c1)
        Core.println("Received:", data)
        first_received_flag = true

        if length(buffer) == buffer_len
            pop!(buffer)
        end
            
        pushfirst!(buffer, data)
    end

    end

    while true
        while length(buffer) ==0
            sleep(0.01)
        end
            
        data = pop!(buffer)
        Core.println("Popped data:", data)
        sleep(0.2)

        if length(buffer) == 0
            break
        end
        
        # @show buffer
    end
end

c1 = Channel{Int64}(10)

Channel{Int64}(10) (empty)

In [66]:
flag = Observable{Bool}(true)

receiver_task = @tspawnat 2 receiver(c1, flag)

Task (runnable) @0x000000016bc106a0

Waiting to receive data


In [67]:

sender_task = @tspawnat 1 sender(c1)

sleep(0.5)


Waiting to send data
Received:1
Popped data:1
Received:2
Received:3
Popped data:2
Received:4
Received:5
Popped data:3
Received:6
Popped data:4
Received:7
Received:8
Popped data:5
Received:9
Received:10
Popped data:6
Popped data:7
Popped data:8
Popped data:9
Popped data:10
